<a href="https://colab.research.google.com/github/hexpert/Equlibrum/blob/main/224329011_Ahmet_Muratoglu_YZM507_Vize_Proje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

224329011 Ahmet Muratoğlu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
# Görüntüyü yükleme
image = cv2.imread('/content/drive/MyDrive/YZM507/goruntu3.png', cv2.IMREAD_GRAYSCALE)
cv2_imshow(image)

In [ ]:
# 1. Histogramı çizdirme
plt.hist(image.ravel(), 256, [0, 256])
plt.title('Original Image Histogram')
plt.show()

In [ ]:
# 2. Histogram eşitleme ve sonuçları çizdirme
equalized_image = cv2.equalizeHist(image)
plt.hist(equalized_image.ravel(), 256, [0, 256])
plt.title('Equalized Histogram')
plt.show()

In [ ]:
# Görüntüyü gösterme
cv2_imshow(equalized_image)
cv2.waitKey(0)

In [ ]:
# 3. Bicubic interpolasyon ile iki kat büyütme
resized_image = cv2.resize(equalized_image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
plt.hist(resized_image.ravel(), 256, [0, 256])
plt.title('Resized Image Histogram')
plt.show()

In [ ]:
# 4. Unsharp masking ve highboost filtering
gaussian_blur = cv2.GaussianBlur(resized_image, (9, 9), 0)
unsharp_image = cv2.addWeighted(resized_image, 1.5, gaussian_blur, -0.5, 0)
plt.hist(unsharp_image.ravel(), 256, [0, 256])
plt.title('Unsharp Masking Histogram')
plt.show()

In [ ]:
# 5. DFT uygulama ve genlik spektrumunu çizdirme
dft = cv2.dft(np.float32(unsharp_image), flags=cv2.DFT_COMPLEX_OUTPUT)
dft_shift = np.fft.fftshift(dft)
magnitude_spectrum = 20 * np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]))
#plt.show(magnitude_spectrum, cmap='gray')
#plt.show(magnitude_spectrum, cmap="gray")
plt.title('Magnitude Spectrum')
plt.show()
cv2_imshow(magnitude_spectrum)
cv2.waitKey(0)

In [ ]:
# 6. Gauss alçak geçiren filtre uygulama
rows, cols = unsharp_image.shape
crow, ccol = rows // 2 , cols // 2
mask = np.zeros((rows, cols, 2), np.uint8)
mask[crow-200:crow+200, ccol-200:ccol+200] = 1

In [ ]:
# DFT sonucu ile maskeyi çarparak alçak geçiren filtreyi uyguluyoruz
fshift = dft_shift * mask
f_ishift = np.fft.ifftshift(fshift)
img_back = cv2.idft(f_ishift)
img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

In [ ]:
# 7. Görüntünün ters DFT'sini alıp son halini çizdirme
cv2_imshow(img_back, cmap='gray')
plt.title('Final Image after Inverse DFT')
plt.show()